In [1]:
import pandas as pd
from pandas import Series,DataFrame

In [2]:
generate='./data/generate/'
rec_path='./data/rec_data/'
train=pd.read_csv(rec_path+'train-dataset.csv')
test=pd.read_csv(rec_path+'test-dataset.csv')
rec_train=pd.read_csv(rec_path+'rec-train-dataset.csv')

In [3]:
trian=train[train['label']==1]
train=train[['suv','userSeq']]
rec_train=rec_train[rec_train['label']==1]
rec_train=rec_train[['suv','userSeq']]
test=test[['suv','userSeq']]

data=pd.concat([train,test])
data=pd.concat([data,rec_train])
# data=train.copy()

data.drop_duplicates(subset=['suv','userSeq'],inplace=True)
data=data[data['userSeq'].notna()]

In [4]:
del train
del test
del rec_train

In [5]:
suv_list=list()
itemId_list=list()
time_list=list()

In [6]:
for suv,userSeq in zip(data['suv'],data['userSeq']):  
  if type(userSeq)==float:
    continue
  seq_arr=userSeq.split(';')
  for i in seq_arr:
    temp=i.split(':')
    suv_list.append(suv)
    itemId_list.append(temp[0])
    time_list.append(temp[1])

# for suv,itemId,logTs in zip(train['suv'],train['itemId'],train['logTs']):  
#   suv_list.append(suv)
#   itemId_list.append(itemId)
#   time_list.append(logTs)

In [7]:
data_suv_seq={
    'suv':Series(suv_list),
    'itemId':Series(itemId_list),
    'logTs':Series(time_list),
}
df_suv_seq=DataFrame(data_suv_seq)
df_suv_seq.drop_duplicates(inplace=True)
suv_list.clear()
itemId_list.clear()
time_list.clear()

In [8]:
data

,suv,userSeq
0,1581173072343wnwm3q,10094375:1640918979673;10084835:1640836934361;...
15,1622811407723249,10095850:1640986831381;10098478:1640986822552;...
46,1631255051069rUc2F4,10072319:1640596871469;10080567:1640771034283;...
105,15941340556035hgvd5,10091382:1640965592215;10090686:1640965772627;...
118,1532509211790143,10090887:1640878331632;10090686:1640878095316;...
...,...,...
1398963,1641397505962MKVTgo,10137082:1641397836542;10136812:1641397895892;...
1398965,163722269070823d2sz,10127450:1641315043235;10130228:1641301456003;...
1398984,1640622537887022,10123004:1641315161620;10131010:1641313915572;...
1398999,1607954310164fjayp1,10137082:1641365824300;10130460:1641313366798;...


In [9]:
df_suv_seq['length'] = df_suv_seq.groupby(['suv'])['itemId'].transform('count')

df_suv_seq.sort_values(by=['logTs'],ascending=False,inplace=True)
df_suv_seq=df_suv_seq.drop_duplicates(keep=False)

In [10]:
df_suv_seq.to_csv(generate+'user_seq.csv',index=False)